In [13]:
import numpy as np
import random
import torch
import torch.nn as nn
from collections import deque
from mpe2 import simple_tag_v3
import supersuit as ss

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
import random
from collections import deque
import numpy as np
import time

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)


class ReplayBuffer:
    def __init__(self, size):
        self.buffer = deque(maxlen=size)

    def add(self, *transition):
        self.buffer.append(transition)

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            torch.as_tensor(np.stack(states), dtype=torch.float32),
            torch.as_tensor(actions, dtype=torch.long),
            torch.as_tensor(rewards, dtype=torch.float32),
            torch.as_tensor(np.stack(next_states), dtype=torch.float32),
            torch.as_tensor(dones, dtype=torch.float32)
        )

    def __len__(self):
        return len(self.buffer)


class DQN:
    def __init__(self,
            env,
            learning_rate = 1e-3,
            buffer_size = 300000,
            learning_starts = 100,
            batch_size = 32,
            tau = 1.0,
            gamma = 0.99,
            train_freq = 4,
            gradient_steps = 1,
            target_update_interval = 10000,
            exploration_fraction = 0.1,
            exploration_initial_eps = 1.0,
            exploration_final_eps = 0.05,
            max_grad_norm = 10,
            verbose = 0
        ):
        
        self.env = env
        self.device = torch.device("mps" if torch.mps.is_available() else "cpu")
        example_agent = env.agents[0]
        self.obs_dim = env.observation_space(example_agent).shape[0]
        self.n_actions = env.action_space(example_agent).n

        self.policy = MLP(self.obs_dim, self.n_actions).to(self.device)
        self.target_policy = MLP(self.obs_dim, self.n_actions).to(self.device)
        self.target_policy.load_state_dict(self.policy.state_dict())
        self.target_policy.eval()

        self.optimizer = optim.Adam(self.policy.parameters(), lr=learning_rate, betas=(0.9, 0.999))
        self.replay_buffer = ReplayBuffer(buffer_size)

        self.learning_starts = learning_starts
        self.batch_size = batch_size
        self.tau = tau
        self.gamma = gamma
        self.train_freq = train_freq
        self.gradient_steps = gradient_steps
        self.target_update_interval = target_update_interval
        self.exploration_initial_eps = exploration_initial_eps
        self.exploration_final_eps = exploration_final_eps
        self.exploration_fraction = exploration_fraction
        self.max_grad_norm = max_grad_norm
        self.verbose = verbose

        self.epsilon = self.exploration_initial_eps
        self.last_loss = None
        self.num_updates = 0


    def _select_action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.n_actions)
        state = torch.FloatTensor(state).unsqueeze(0).to(self.device)
        with torch.no_grad():
            act_values = self.policy(state)
        return torch.argmax(act_values[0]).item()

    def _train_step(self):
        if len(self.replay_buffer) < self.batch_size:
            return

        self.policy.train()

        total_loss = 0
        for _ in range(self.gradient_steps):
            states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
            states = states.to(self.device)
            actions = actions.to(self.device)
            rewards = rewards.to(self.device)
            next_states = next_states.to(self.device)
            dones = dones.to(self.device)

            q_values = self.policy(states).gather(1, actions.unsqueeze(1)).squeeze(1)

            with torch.no_grad():
                next_q_values = self.target_policy(next_states).max(1)[0]
                expected_q = rewards + self.gamma * next_q_values * (1 - dones)

            loss = F.smooth_l1_loss(q_values, expected_q)
            self.last_loss = loss.item()

            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(self.policy.parameters(), self.max_grad_norm)
            self.optimizer.step()

            self.num_updates += 1

            total_loss += loss.item()
        
        return total_loss / self.gradient_steps

    def learn(self, total_timesteps, log_interval=1000):
        obs, _ = self.env.reset()
        state = np.asarray(obs, dtype=np.float32)
        start_t  = time.perf_counter()
        ep_count = 0
        ep_lens  = deque(maxlen=100)
        ep_rews  = deque(maxlen=100)
        episode_reward = 0
        eps_decay_steps = int(self.exploration_fraction * total_timesteps)

        for t in range(1, total_timesteps + 1):
            self.total_timesteps = t
            self.epsilon = max(
                self.exploration_final_eps,
                self.exploration_initial_eps - (self.exploration_initial_eps - self.exploration_final_eps) * (t / eps_decay_steps)
            )

            action = self._select_action(state)
            next_state, reward, terminated, truncated, info = self.env.step(action)
            #reward = np.clip(reward, -1.0, 1.0)
            done = terminated or truncated
            next_state = np.asarray(next_state, dtype=np.float32)
            self.replay_buffer.add(state, action, reward, next_state, float(done))
            state = next_state
            episode_reward += reward

            if "episode" in info:
                ep_len = info["episode"]["l"]
                ep_rew = info["episode"]["r"]
                ep_lens.append(ep_len)
                ep_rews.append(ep_rew)
                ep_count += 1

            ## Fin del episodio
            if done:
                last_ep_reward = episode_reward
                obs, _ = self.env.reset()
                state = np.asarray(obs, dtype=np.float32)
                episode_reward = 0

            ## Cuándo se entrena la red
            if t > self.learning_starts and t % self.train_freq == 0:
                self._train_step()

            ## Sincronización de red target
            if self.num_updates > 0 and self.num_updates % self.target_update_interval == 0:
                self.target_policy.load_state_dict(self.policy.state_dict())
                self.target_policy.eval()

            if self.verbose and ep_count > 0 and ep_count % log_interval == 0 and done:
                fps = int(t / (time.perf_counter() - start_t))
                lr = self.optimizer.param_groups[0]["lr"]
                loss = self.last_loss if self.last_loss is not None else float("nan")
                print("----------------------------------")
                print(f"| rollout/            |          |")
                print(f"|    ep_len_mean      | {np.mean(ep_lens) if ep_lens else float('nan'):.0f}")
                print(f"|    ep_rew_mean      | {np.mean(ep_rews) if ep_rews else float('nan'):.0f}")
                print(f"|    exploration_rate | {self.epsilon:.3f}")
                print(f"| time/               |          |")
                print(f"|    episodes         | {ep_count}")
                print(f"|    fps              | {fps}")
                print(f"|    time_elapsed     | {int(time.perf_counter() - start_t)}")
                print(f"|    total_timesteps  | {t}")
                print(f"| train/              |          |")
                print(f"|    learning_rate    | {lr:.3g}")
                print(f"|    loss             | {loss:.4f}")
                print(f"|    n_updates        | {self.num_updates}")
                print("----------------------------------")

In [15]:
def learn_multiagent(env, agent_dict, total_timesteps, log_interval=1000):
    observations, _ = env.reset()
    episode_reward = {agent: 0 for agent in env.agents}
    ep_lens = deque(maxlen=100)
    ep_len = 0
    ep_rews = {agent: deque(maxlen=100) for agent in agent_dict}
    ep_count = 0
    t = 0
    start_t = time.perf_counter()

    eps_decay_steps = int(list(agent_dict.values())[0].exploration_fraction * total_timesteps)

    episode_rewards = []
    losses = []

    last_logged = -1

    while t < total_timesteps:
        actions = {}

        total_rewards = {agent: 0 for agent in env.agents}
        total_losses = {agent: 0 for agent in env.agents}

        for agent in env.agents:
            if agent in agent_dict:
                dqn = agent_dict[agent]
                dqn.epsilon = max(
                    dqn.exploration_final_eps,
                    dqn.exploration_initial_eps - (dqn.exploration_initial_eps - dqn.exploration_final_eps) * (t / eps_decay_steps)
                )
                action = dqn._select_action(observations[agent])
                actions[agent] = action
            else:
                actions[agent] = 0  # good agent se queda quieto

        next_obs, rewards, terminations, truncations, infos = env.step(actions)

        for agent in agent_dict:
            done = terminations[agent] or truncations[agent]
            dqn = agent_dict[agent]

            dqn.replay_buffer.add(
                np.array(observations[agent], dtype=np.float32),
                actions[agent],
                rewards[agent],
                np.array(next_obs[agent], dtype=np.float32),
                float(done)
            )
            episode_reward[agent] += rewards[agent]

            if t > dqn.learning_starts and t % dqn.train_freq == 0:
                loss = dqn._train_step()
                total_losses[agent] += loss if loss is not None else 0

            if dqn.num_updates > 0 and dqn.num_updates % dqn.target_update_interval == 0:
                dqn.target_policy.load_state_dict(dqn.policy.state_dict())
                dqn.target_policy.eval()

        losses.append(total_losses)

        t += 1
        observations = next_obs
        ep_len += 1
        # Fin de episodio
        if terminations["agent_0"] or truncations["agent_0"]:
            episode_rewards.append(episode_reward.copy())

            for agent in agent_dict:
                ep_rews[agent].append(episode_reward[agent])
                episode_reward[agent] = 0
            
            ep_lens.append(ep_len)  # si no tienes env.env_done_step puedes usar len(env.dones)
            ep_len = 0
            observations, _ = env.reset()
            ep_count += 1

        # Logging
        if ep_count > 0 and ep_count % log_interval == 0 and ep_count != last_logged:
            last_logged = ep_count
            fps = int(t / (time.perf_counter() - start_t))
            print("----------------------------------")
            print(f"| time/               |          |")
            print(f"|    episodes         | {ep_count}")
            print(f"|    fps              | {fps}")
            print(f"|    time_elapsed     | {int(time.perf_counter() - start_t)}")
            print(f"|    total_timesteps  | {t}")
            print(f"|    ep_len_mean      | {np.mean(ep_lens) if ep_lens else float('nan'):.0f}")
            for agent in agent_dict:
                dqn = agent_dict[agent]
                loss = dqn.last_loss if dqn.last_loss is not None else float("nan")
                print(f"| {agent} -> mean_ep_rew | {np.mean(ep_rews[agent]):.2f} | loss: {loss:.4f} | eps: {dqn.epsilon:.3f}")
            print("----------------------------------")

    return episode_rewards, losses

In [16]:
import matplotlib.pyplot as plt

def plot_total_rewards(rewards_dict_per_episode):
    """
    rewards_dict_per_episode: list de dicts por episodio, ej:
    [{agent_0: 10, agent_1: 12}, {agent_0: 5, agent_1: 7}, ...]
    """

    # llevar a cpu
    agent_names = rewards_dict_per_episode[0].keys()
    episode_count = len(rewards_dict_per_episode)

    for agent in agent_names:
        rewards = [episode[agent].cpu().item() if torch.is_tensor(episode[agent]) else episode[agent] for episode in rewards_dict_per_episode]
        plt.plot(rewards, label=agent)

    plt.xlabel("Episodio")
    plt.ylabel("Reward total por agente")
    plt.title("Recompensas por episodio")
    plt.legend()
    plt.grid(True)
    plt.show()

In [17]:
import pandas as pd

In [18]:
# Crear entorno paralelo
seed = 42
env = simple_tag_v3.parallel_env(render_mode='rgb_array', num_good=1, num_adversaries=3, num_obstacles=0, max_cycles=150, dynamic_rescaling = True)
env.reset(seed=seed)
agents = env.agents

dqn_params = {
    "learning_rate": 1e-3,
    "buffer_size": 1000000,
    "learning_starts": 5000,
    "batch_size": 128,
    "tau": 1.0,
    "gamma": 0.99,
    "train_freq": 16,
    "gradient_steps": 4,
    "target_update_interval": 2000,
    "exploration_fraction": 0.1,
    "exploration_initial_eps": 1,
    "exploration_final_eps": 0.1,
    "max_grad_norm": 10,
    "verbose": 1,
}

# Inicializar agentes solo para los adversarios
for i in range(1, 16):
    agent_dict = {
        agent: DQN(env, **dqn_params)
        for agent in agents if agent.startswith("adversary")
    }

    # Compartir replay buffer
    #-------------
    shared_buffer = agent_dict["adversary_0"].replay_buffer
    for name in agent_dict:
        agent_dict[name].replay_buffer = shared_buffer
    #-------------

    # Entrenar los agentes adversarios
    episode_rewards, episode_losses = learn_multiagent(env, agent_dict, total_timesteps=200000, log_interval=100)

    filename = f"runs_multi_dqn_replay_buffer/multi_dqn_replay_buffer_run_{i}.csv"
    cleaned_log = []

    for ep_idx, ep in enumerate(episode_rewards):
        row = {"episode": ep_idx}

        for agent, value in ep.items():
            if torch.is_tensor(value):
                value = value.cpu().item()
            row[agent] = value

        cleaned_log.append(row)

    # Crear DataFrame
    df = pd.DataFrame(cleaned_log)
    df.to_csv(filename, index=False)


----------------------------------
| time/               |          |
|    episodes         | 100
|    fps              | 412
|    time_elapsed     | 36
|    total_timesteps  | 15000
|    ep_len_mean      | 150
| adversary_0 -> mean_ep_rew | 9.60 | loss: 0.0006 | eps: 0.325
| adversary_1 -> mean_ep_rew | 9.60 | loss: 0.0002 | eps: 0.325
| adversary_2 -> mean_ep_rew | 9.60 | loss: 0.0747 | eps: 0.325
----------------------------------
----------------------------------
| time/               |          |
|    episodes         | 200
|    fps              | 313
|    time_elapsed     | 95
|    total_timesteps  | 30000
|    ep_len_mean      | 150
| adversary_0 -> mean_ep_rew | 5.30 | loss: 0.0762 | eps: 0.100
| adversary_1 -> mean_ep_rew | 5.30 | loss: 0.0016 | eps: 0.100
| adversary_2 -> mean_ep_rew | 5.30 | loss: 0.2311 | eps: 0.100
----------------------------------
----------------------------------
| time/               |          |
|    episodes         | 300
|    fps              | 28